<a href="https://colab.research.google.com/github/mikislin/CNE25/blob/main/notebooks/CNE_Class2_Stats_Exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instructions: How to Complete and Submit Class 2 Exercise

1. Save a Copy to Your Google Drive

Go to File → Save a copy in Drive - This creates your personal editable version.

2. Complete the Assignment

Write your code in the cells labeled #solution. Write your written answers or explanations in the text (Markdown) cells.

3. Save Your Work

Your edits are automatically saved to your Drive, but double-check before submitting.

4. Share Your Notebook

When you’re finished, click Share (top-right corner). Set sharing to “Anyone with the link can view.” DM your notebook link to me on Slack

# Problem 1:  Standard Deviation (SD) vs. Standard Error of the Mean (SEM) - Conceptual Interpretation



Use the interactive "Standard Deviation (SD) vs. Standard Error of the Mean (SEM)" simulation in [class notebook](https://colab.research.google.com/github/mikislin/CNE25/blob/main/CNE_Class2_Stats.ipynb)

Instructions:

1. Run the Simulation: Execute the code block containing the "SD vs. Standard Error of the Mean (SEM)" simulation.

2. Manipulate Parameters:
  Interact with the simulation by performing the following actions:

    Set the Sample Size (n) to a small value (e.g., 10). Click the "Draw 100 Samples" button multiple times.

    Now, set the Sample Size (n) to a large value (e.g., 500). Click the "Draw 100 Samples" button multiple times.

    Click the "Reset Simulation" button.

**(a)**  Imagine (Simulate) the "Population" represents the firing rate of a specific neuron in response to a stimulus, recorded over thousands of trials (mean rate = 20Hz). The "Population SD" is 7.2 and reflects the biological variability of this neuron's response. You are conducting an experiment where you can only record 20 trials (n=20).

In [ ]:
# solution

If you report the **mean firing rate ± SD**, what are you communicating about the neuron's behavior?


reply here:

If you instead report the **mean firing rate ± SEM**, what are you communicating about the precision of your estimate of the mean?


reply here:

A colleague reviews your paper and says -

> You should report SEM because the error bars are smaller for 20 recorded trials before and after treatment, which makes the figure look better.

Based on the simulation, provide a principled argument when it is appropriate to use SD versus SEM on a graph for group comparison.


In [ ]:
# solution

# Problem 2:  Automating Statistical Decisions and Critical Analysis

Instructions:

  **(a)**. Review the compare_groups function: Analyze its logic.


```
def compare_groups(df, metric_col, group_col='Group'):
    groups = df[group_col].unique()
    if len(groups) != 2:
        raise ValueError("This function is designed for two-group comparison.")

    # Ensure consistent group order for reporting
    group1_name = "WT" if "WT" in groups else groups[0]
    group2_name = "KO" if "KO" in groups else groups[1]

    group1_data = df[df[group_col] == group1_name][metric_col].dropna()
    group2_data = df[df[group_col] == group2_name][metric_col].dropna()

    if len(group1_data) < 3 or len(group2_data) < 3:
        # Shapiro test requires at least 3 samples
        is_normal = False
        norm1_p, norm2_p = np.nan, np.nan
    else:
        _, norm1_p = stats.shapiro(group1_data) #checks for normality (Shapiro-Wilk test)
        _, norm2_p = stats.shapiro(group2_data)
        is_normal = norm1_p > 0.05 and norm2_p > 0.05

    _, var_p = stats.levene(group1_data, group2_data) #homogeneity of variances (Levene's test)
    is_equal_variance = var_p > 0.05

    if is_normal:
        if is_equal_variance:
            test_name = "Independent T-test"
            stat, p_val = stats.ttest_ind(group1_data, group2_data, equal_var=True)
        else:
            test_name = "Welch's T-test"
            stat, p_val = stats.ttest_ind(group1_data, group2_data, equal_var=False)
    else:
        test_name = "Mann-Whitney U"
        # The smaller of the two U values is typically reported
        stat, p_val = stats.mannwhitneyu(group1_data, group2_data, alternative='two-sided')

    return {
        'Metric': metric_col,
        'Test Used': test_name,
        'Statistic': f"{stat:.3f}",
        'P-value': f"{p_val:.4f}",
        'Significant': p_val < 0.05,
        f'Mean_{group1_name}': f"{group1_data.mean():.2f}",
        f'Median_{group1_name}': f"{group1_data.median():.2f}", # ADDED
        f'Std_{group1_name}': f"{group1_data.std():.2f}",
        f'N_{group1_name}': len(group1_data),
        f'Mean_{group2_name}': f"{group2_data.mean():.2f}",
        f'Median_{group2_name}': f"{group2_data.median():.2f}", # ADDED
        f'Std_{group2_name}': f"{group2_data.std():.2f}",
        f'N_{group2_name}': len(group2_data),
        'Shapiro_p (G1)': f"{norm1_p:.3f}",
        'Shapiro_p (G2)': f"{norm2_p:.3f}",
        'Levene_p': f"{var_p:.3f}",
    }
```



**(b)** Create and Analyze a Simulated Dataset (as pd.DataFrame)

Group: 'Control' and 'Treatment'

metric_col:

  Synaptic_Density: A normally distributed variable with different means for each group.

  Behavioral_Score: A non-normally distributed variable. Make the variance in 'Treatment' much larger than in 'Control' for this metric.

  Neuron_FRate: Data that is approximately normal.

In [ ]:
# solution

**(c)** What are limitations of a fully automated decision function like `compare_groups`?

Consider scenarios where the Shapiro-Wilk p-value is close to the 0.05 threshold (e.g., 0.049 or 0.051).

What role does researcher judgment play in such cases? How to get the input from the researcher for marginal cases?

In [ ]:
# solution

Bonus: How would you adapt your analysis for this three-group design?

In [ ]:
# solution

# Problem 3:  ANOVA vs General Linear Model (specifically, a Linear Mixed-Effects Model)  

**Scenario:**

You are analyzing data from a new drug's effect on dendritic spine density in a mouse model of a neurological disorder.

*The experiment design.*

  Fixed Factors (what you are testing):
1. genotype: Wild-Type (WT) vs. Knock-Out (KO)
2. treatment: Vehicle (placebo) vs. Drug
3. sex: Male vs. Female

Nuisance/Grouping Variables (sources of variability to control for):

1. mouse_id: Multiple measurements (e.g., from different brain regions) are taken from the same mouse. These measurements are not independent.

2. cohort: The experiment was run in several batches or "cohorts" over many weeks. There might be slight environmental or procedural differences between cohorts.

Your goal is to determine if the drug works, and if its effect depends on the genotype or sex of the animal, while properly accounting for the data structure.

**(a)**   generate a realistic dataset

with initial parameters and weight of preductors

```
# Define the "true" effects
baseline = 5.0
genotype_effect = -1.0 if genotype == 'KO' else 0
treatment_effect = 0.8 if treatment == 'Drug' else 0
sex_effect = +0.2 if sex == 'Female' else 0
# CRUCIAL INTERACTION: Drug works better in KO mice
interaction_effect = 1.5 if (genotype == 'KO' and treatment == 'Drug') else 0

# Combine all effects + random noise
spine_density = (baseline + genotype_effect + treatment_effect +  interaction_effect + mouse_effect + cohort_effect + np.random.normal(0, 0.5))
```



In [ ]:
# Use the following experiment parameters for Data Simulation
np.random.seed(42)
n_mice_per_group = 12
n_measurements_per_mouse = 5
genotypes = ['WT', 'KO']
treatments = ['Vehicle', 'Drug']
sexes = ['Male', 'Female']
cohorts = ['C1', 'C2', 'C3']

data = []
mouse_counter = 0

In [ ]:
# solution

**(b)** perform Three-Way ANOVA using OLS

In [ ]:
# solution

**(c)** perform analysis with a Linear Mixed-Effects Model

In [ ]:
# solution

**(d)** Review summaries of the ANOVA and LME (GLM).
What does the coefficients tell you about how the drug works?